In [1]:
import tensorflow as tf
import numpy as np
from scipy.io import loadmat
from tensorflow.python.framework import ops

C:\Users\amcle\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
def weight_mapping(theta):
    theta = 1.0 + np.divide(spice_params['K'], 1.0 + np.exp(-theta))
    sum_theta = np.sum(theta, axis=1)
    g = np.divide(theta, np.tile(sum_theta, (theta.shape[1], 1)).transpose())
    # np.tile repeats the row 394 times then transposes to get the right dimension
    return g.astype(np.float32)

def sigmoid(z):
    half_Vdd = -spice_params['Vdd'] / 2.0
    sharp_factor = spice_params['cc'][3]
    s_param = spice_params['cc'][2]
    
    return np.multiply(half_Vdd, np.tanh(np.multiply(sharp_factor, np.subtract(z, s_param))))

def sigmoidn(z):
    half_Vdd = spice_params['Vdd'] / 2.0
    sharp_factor = spice_params['cc2'][3]
    s_param = spice_params['cc2'][2]
    
    return np.multiply(half_Vdd, np.tanh(np.multiply(sharp_factor, np.subtract(z, s_param))))

def sigmoidGradient(z):
    grad = np.multiply(-spice_params['cc'][3], np.subtract(1, np.square(np.tanh(np.multiply(spice_params['cc'][3], \
                                                                        np.subtract(z, spice_params['cc'][2]))))))
    return grad

def sigmoidGradientn(z):
    grad_n = np.multiply(spice_params['cc2'][3], np.subtract(1, np.square(np.tanh(np.multiply(spice_params['cc2'][3], \
                                                                        np.subtract(z, spice_params['cc2'][2]))))))
    return grad_n

In [3]:
# Is not really considered a function we should care about for backprop to work correctly 
def randInitializeWeights(L_in,L_out):
    '''Function that randomly initializes the weights of a layer with L_in incoming connections
    and L_out outgoing connections as implemented in randInitializeWeights.m
    Note that W should be set to a matrix of size(L_out, 1 + L_in) as the column row of W handles 
    the "bias" terms. Duplicates functionality from randInitializeWeights.m 
    Parameters
    ----------
    L_in := tensor, number of incoming connections
    L_out := tensor, number of outgoing connections
    Results returns a tensor with of initial weights for the network'''
    
    epsilon = np.sqrt(6/(L_in+L_out))
    return tf.random_uniform([L_out, L_in + 1], dtype = tf.float32, minval = -np.sqrt(6/(L_in+L_out)), maxval = np.sqrt(6/(L_in+L_out)))
    
def weight_md(theta):
    '''Implements physical circuit constraints on the weight mapping that occurs during backpropogation of the network. This
    function duplicates the functionality of the weight_md.m file.
    Parameters
    ----------
    theta := unconstrained weights matrix
    K := 𝐾 is a constant number which is the difference between the 𝜎𝑚𝑎𝑥 and 𝜎𝑚𝑖𝑛. 
    Results
    ---------
    returns activated neuron output
    ''' 
    num_pn = theta.get_shape().as_list()[0]
    num_n = theta.get_shape().as_list()[1]
    g1 = tf.multiply(spice_params['K'],tf.multiply(tf.divide(1.0,tf.add(1.0,tf.exp(-theta))), \
                                                   tf.subtract(1.0,tf.divide(1.0,tf.add(1.0,tf.exp(-theta))))))
    g1 = tf.transpose(tf.contrib.kfac.utils.kronecker_product(g1,tf.ones((num_n, 1))))
    theta = tf.add(1.0, tf.divide(spice_params['K'],tf.add(1.0,tf.exp(-theta))))
    sum_theta = tf.reduce_sum(theta, 1, keepdims = True)
    sums_theta1 = tf.tile(sum_theta,(1,num_n))
    theta = tf.transpose(tf.contrib.kfac.utils.kronecker_product(theta,tf.ones((num_n, 1))))
    sums_theta = tf.transpose(tf.contrib.kfac.utils.kronecker_product(sums_theta1,tf.ones((num_n, 1))))
    
    # Didn't need to do the extra transpose 
    sums_theta1 = tf.divide(1.0,tf.reshape(sums_theta1, [-1]))
    
    # Calculates the indexes we need from the g array 
    indices = [[i%num_n,i] for i in range(num_n*num_pn)]

    g = tf.divide(-theta, tf.square(sums_theta))
    
    # The updated weights for the network
    updated_weights = tf.add(tf.gather_nd(g, indices), sums_theta1)

    # Indicates a sparse tensor 
    delta = tf.SparseTensor(indices, updated_weights, g.get_shape().as_list())
    
    g = tf.sparse_add(g,delta)
   
    g = tf.multiply(g,g1)
    
    return g

In [4]:
# Makes the function a numpy function 
np_sigmoid = np.vectorize(sigmoid)
#np_weight_map = np.vectorize(weight_mapping)
np_sigmoidGradient = np.vectorize(sigmoidGradient)
np_sigmoidn = np.vectorize(sigmoid)
np_sigmoidGradientn = np.vectorize(sigmoidGradient)

# Make sure the functions are in float32
np_sigmoid_32 = lambda x: np_sigmoid(x).astype(np.float32)
#np_weight_map_32 = lambda x: np_weight_map(x).astype(np.float32)
np_sigmoidGradient_32 = lambda x: np_sigmoidGradient(x).astype(np.float32)
np_sigmoidn_32 = lambda x: np_sigmoidn(x).astype(np.float32)
np_sigmoidGradientn_32 = lambda x: np_sigmoidGradientn(x).astype(np.float32)

In [5]:
def tf_sigmoidGradient(x,name=None):
    with ops.op_scope([x], name, "sigmoidGradient") as name:
        y = tf.py_func(np_sigmoidGradient_32,
                        [x],
                        [tf.float32],
                        name=name,
                        stateful=False)
        return y[0]
    
def tf_sigmoidGradientn(x,name=None):
    with ops.op_scope([x], name, "sigmoidGradientn") as name:
        y = tf.py_func(np_sigmoidGradientn_32,
                        [x],
                        [tf.float32],
                        name=name,
                        stateful=False)
        return y[0]

# Function that tells tensorflow how to calculate the gradients
def py_func(func, inp, Tout, stateful=True, name=None, grad=None):

    # Need to generate a unique name to avoid duplicates:
    rnd_name = 'PyFuncGrad' + str(np.random.randint(0, 1E+8))

    tf.RegisterGradient(rnd_name)(grad)  # see _MySquareGrad for grad example
    g = tf.get_default_graph()
    with g.gradient_override_map({"PyFunc": rnd_name}):
        return tf.py_func(func, inp, Tout, stateful=stateful, name=name)
    
def sigmoidGradient(op, grad):
    x = op.inputs[0]
    n_gr = tf_sigmoidGradient(x)
    return grad * n_gr

def sigmoidGradientn(op, grad):
    x = op.inputs[0]
    n_gr = tf_sigmoidGradientn(x)
    return grad * n_gr

def weightGradient(op, grad):
    x = op.inputs[0]
    n_gr = weight_md(x)
    return grad * n_gr  

def tf_sigmoid(x, name=None):
    
    with tf.name_scope(name, "sigmoid", [x]) as name:
        y = py_func(np_sigmoid_32,
                        [x],
                        [tf.float32],
                        name=name,
                        grad=sigmoidGradient)  # <-- here's the call to the gradient
        return y[0]
    
def tf_sigmoidn(x, name=None):

    with tf.name_scope(name, "sigmoidn", [x]) as name:
        y = py_func(np_sigmoidn_32,
                        [x],
                        [tf.float32],
                        name=name,
                        grad=sigmoidGradientn)  # <-- here's the call to the gradient
        return y[0]
    
def tf_weight_map(x, name=None):

    with tf.name_scope(name, "weight_mapping", [x]) as name:
        y = py_func(weight_mapping,
                        [x],
                        [tf.float32],
                        name=name,
                        grad=weightGradient)  # <-- here's the call to the gradient
        return y[0]

In [6]:
# Dictionary that holds all the Hspice params necessary for this network
spice_params = {'K': 7.33,
               'Vdd': 1.0,
               'cc': np.array([-0.0012, -0.2483,  -0.0235,  31.7581]),
               'cc2': np.array([-0.0011,  0.2490,  -0.0203, 193.0602]),
               'lambda': 0,
               'Ron': 100,
               'Roff': 16e3
               }

# Loading the image input files needed to feed into the network
train_vals = np.random.permutation(np.arange(1000))[:800] # Line 99: main.m
test_vals = np.array([i for i in range(1000) if i not in train_vals])
mnist = loadmat('MNIST_complete.mat') # Line 100: main.m

X_dat = mnist['activationsPooled'] # Line 101: main.m
Y_dat = mnist['y'].transpose() # Line 102: main.m

# Values to Train the network
X_train = ((X_dat[:,train_vals] - 0.5 ) /2.0).transpose() # Line 103: main.m
Y_train = Y_dat[:,train_vals].transpose() # Line 104: main.m

#Values to Test the network
X_test = ((X_dat[:,test_vals] - 0.5 ) /2.0).transpose() # Line 103: main.m
Y_test = Y_dat[:,test_vals].transpose() # Line 104: main.m

# Network Parameters
input_layer_size = 196 # MNIST data input (img shape: 14*14)
num_labels = 10 # MNIST total classes (0-9 digits)
num_hidden_layer = 1 # Line 126: main.m (number of hidden layers)
hidden_layer_size = 100 # Line 127: main.m (1st layer number of neurons)

In [7]:
print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
print(Y_train)

(800, 196) (200, 196) (800, 10) (200, 10)
[[0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [8]:
# tf Graph Input
X = tf.placeholder(tf.float32, [None, 196]) # mnist data image of shape 14*14 = 196
Y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes

# Initializing the Cost J
#J = tf.Variable(tf.zeros([1,]))
J = 0

# Get the number of rows in the fed value at run-time (for our case 800)
m = tf.shape(X)[0]

# Defines the positive and negative bias terms respectively (pbias,nbias)
pbias = tf.fill((m, 1),(spice_params['Vdd']/2.0))
nbias = tf.fill((m, 1),(-spice_params['Vdd']/2.0))

X1 = tf.concat([pbias, X], 1)
X2 = tf.concat([nbias, -X], 1)

# # Defines the trainable parameters
# theta1 = tf.Variable(tf.zeros([hidden_layer_size, input_layer_size + 1]))
# theta1n = tf.Variable(tf.zeros([hidden_layer_size, input_layer_size + 1]))
# theta2 = tf.Variable(tf.zeros([num_labels, hidden_layer_size + 1]))
# theta2n = tf.Variable(tf.zeros([num_labels, hidden_layer_size + 1]))

# Defining the unconstrained weights
nn_params = {
    'Theta1': tf.Variable(tf.zeros([hidden_layer_size, input_layer_size + 1])),
    'Theta1n': tf.Variable(tf.zeros([hidden_layer_size, input_layer_size + 1])),
    'Theta2': tf.Variable(tf.zeros([num_labels, hidden_layer_size + 1])),
    'Theta2n': tf.Variable(tf.zeros([num_labels, hidden_layer_size + 1])),
}
for key in nn_params:
    print(key, nn_params[key].get_shape().as_list())

Theta1 [100, 197]
Theta1n [100, 197]
Theta2 [10, 101]
Theta2n [10, 101]


In [9]:
# Initializes the delta of weights and biases of network
Delta_2 = tf.zeros(tf.shape(nn_params['Theta2']))
Delta_1 = tf.zeros(tf.shape(nn_params['Theta1']))
delta_3 = tf.zeros([num_labels,1])
delta_2 = tf.zeros([hidden_layer_size,1])
    
# Forward propogating through the network with the mapped constrained weights 
weights1 = tf_weight_map(tf.concat([nn_params['Theta1'],nn_params['Theta1n']], 1)) # constrained weights
    
sigma1 = tf.add(tf.matmul(X1,tf.transpose(weights1[:tf.shape(nn_params['Theta1'])[0],:tf.shape(nn_params['Theta1'])[1]])), \
                tf.matmul(X2,tf.transpose(weights1[:tf.shape(nn_params['Theta1'])[0],tf.shape(nn_params['Theta1'])[1]:])))
    
h1 = tf_sigmoid(sigma1) # Activation values out of hidden layer
    
weights2 = tf_weight_map(tf.concat([nn_params['Theta2'],nn_params['Theta2n']], 1)) # constrained weights
    
sigmaO = tf.add(tf.matmul(tf.concat([pbias,h1],1),tf.transpose(weights2[:tf.shape(nn_params['Theta2'])[0],:tf.shape(nn_params['Theta2'])[1]])), \
                tf.matmul(tf.concat([nbias,tf_sigmoidn(sigma1)],1),tf.transpose(weights2[:tf.shape(nn_params['Theta2'])[0],tf.shape(nn_params['Theta2'])[1]:])))
  
# Activation values out of the output layer
h_theta = tf.add(tf_sigmoid(sigmaO), 0.5)

# Loss calculated per Arash recommendation
J_h = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = h_theta, labels = Y))

# Forward Propogation Step
a_1 = tf.concat([tf.transpose(pbias), tf.transpose(X)], 0)
a_1n = tf.concat([tf.transpose(nbias), tf.transpose(-X)], 0)
z_2 = tf.add(tf.matmul(weights1[:tf.shape(nn_params['Theta1'])[0],:tf.shape(nn_params['Theta1'])[1]], a_1),
                    tf.matmul(weights1[:tf.shape(nn_params['Theta1'])[0],tf.shape(nn_params['Theta1'])[1]:], a_1n))

a_2 = tf.concat([tf.transpose(pbias), tf_sigmoid(z_2)], 0)
a_2n = tf.concat([tf.transpose(nbias), tf_sigmoidn(z_2)], 0)
z_3 = tf.add(tf.matmul(weights2[:tf.shape(nn_params['Theta2'])[0],:tf.shape(nn_params['Theta2'])[1]],a_2),
                    tf.matmul(weights2[:tf.shape(nn_params['Theta2'])[0],tf.shape(nn_params['Theta2'])[1]:],a_2n))
#a_3 = tf_sigmoid(z_3)
a_3 = tf.add(tf_sigmoid(z_3), 0.5)

# Loss calculated per Arash recommendation
J_a = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = tf.transpose(a_3), labels = Y))


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In the cell above J_a and J_h should have the same value of cost, wondering if we can get away with only calculating one. If you run all the cells besides the two below than you can test this functionality for yourself 

In [ ]:
weights1_md = tf.gradients(weights1, [tf.concat([nn_params['Theta1'], nn_params['Theta1n']], 1)])[0]
weights2_md = tf.gradients(weights2, [tf.concat([nn_params['Theta2'], nn_params['Theta2n']], 1)])[0]

In [ ]:
# Back Propogation Step
delta_3 = -tf.subtract(a_3, tf.transpose(Y)) # Gets the difference 
delta_2 = tf.add(tf.multiply(tf.matmul(tf.transpose(weights2[:tf.shape(nn_params['Theta2'])[0],:tf.shape(nn_params['Theta2'])[1]]), delta_3), \
                                        tf.concat([tf.transpose(pbias), sigmoidGradient(z_2)], 0)),\
                            tf.multiply(tf.matmul(tf.transpose(weights2[:tf.shape(nn_params['Theta2'])[0],:tf.shape(nn_params['Theta2'])[1]]), delta_3), \
                                        tf.concat([tf.transpose(nbias), sigmoidGradientn(z_2)], 0)))    

# # Re-Mapping the unconstrianed weights 
# weights1_md = weight_md(tf.concat([nn_params['Theta1'], nn_params['Theta1n']], 1))
# weights2_md = weight_md(tf.concat([nn_params['Theta2'], nn_params['Theta2n']], 1))

# Re-Mapping the unconstrianed weights 
#tf.gradients(y, [x])[0]
weights1_md = tf.gradients(weights1, [tf.concat([nn_params['Theta1'], nn_params['Theta1n']], 1)])[0]
weights2_md = tf.gradients(weights2, [tf.concat([nn_params['Theta2'], nn_params['Theta2n']], 1)])[0]

# Total delta from the 2nd layer of network
Delta_2t = tf.matmul(delta_3, tf.matmul(tf.transpose(tf.concat([a_2,a_2n], 0)), weights2_md))

# Calculates indices to gather values from total delta of 2nd layer for positive and negative contributions respectively
Delta2_pidx = []
Delta2_nidx = []
Delta2_row = delta_3.get_shape()[0]
Delta2_col = nn_params['Theta2'].get_shape().as_list()[1]

for r in range(Delta2_row):
    #print('Positive Indices -> ROW: %s Column Range: %d - %d' % (r, r*2*col_size, (2*r+1)*col_size))
    #print('Negative Indices -> ROW: %s Column Range: %d - %d' % (r, (2*r+1)*col_size, (r+1)*2*col_size))
    Delta2_pidx.extend([[r,i] for i in range(r*2*Delta2_col, (2*r+1)*Delta2_col)])
    Delta2_nidx.extend([[r,i] for i in range((2*r+1)*Delta2_col, (r+1)*2*Delta2_col)]) 

# Separates the contributed delta from the positive crossbar and negative crossbar of the 2nd layer
Delta_2 = tf.reshape(tf.gather_nd(Delta_2t, Delta2_pidx),[Delta2_row, -1])
Delta_2n = tf.reshape(tf.gather_nd(Delta_2t, Delta2_nidx),[Delta2_row, -1])
    
# Total delta from the 1st layer of network
Delta_1t = tf.matmul(delta_2[1:,:],tf.matmul(tf.transpose(tf.concat([a_1,a_1n], 0)), weights1_md))

# Calculates indices to gather values from total delta of 1st layer for positive and negative contributions respectively
Delta1_pidx = []
Delta1_nidx = []
Delta1_row = hidden_layer_size
Delta1_col = nn_params['Theta1'].get_shape().as_list()[1]

for r in range(Delta1_row):
    #print('Positive Indices -> ROW: %s Column Range: %d - %d' % (r, r*2*Delta1_col, (2*r+1)*Delta1_col))
    #print('Negative Indices -> ROW: %s Column Range: %d - %d' % (r, (2*r+1)*Delta1_col, (r+1)*2*Delta1_col))
    Delta1_pidx.extend([[r,i] for i in range(r*2*Delta1_col, (2*r+1)*Delta1_col)])
    Delta1_nidx.extend([[r,i] for i in range((2*r+1)*Delta1_col, (r+1)*2*Delta1_col)])
    
# Separates the contributed delta from the positive crossbar and negative crossbar of the 1st layer
Delta_1 = tf.reshape(tf.gather_nd(Delta_1t, Delta1_pidx),[Delta1_row, -1])
Delta_1n = tf.reshape(tf.gather_nd(Delta_1t, Delta1_nidx),[Delta1_row, -1]) 
    
# Theta 2 gradient Variables are updated
theta_coef = tf.multiply(tf.cast(tf.divide(1,m), dtype = tf.float32), tf.constant(spice_params['cc'][3], dtype = tf.float32))

theta2_grad = tf.concat([tf.reshape(tf.multiply(tf.multiply((2.0/spice_params['Vdd']),theta_coef), Delta_2[:,0]), [-1,1]), \
                         tf.add(tf.multiply(tf.multiply((2.0/spice_params['Vdd']),theta_coef), Delta_2[:,1:]), \
                   tf.multiply(nn_params['Theta2'][:,1:], tf.cast(tf.divide(spice_params['lambda'],m), dtype = tf.float32)))], 1)
theta2n_grad = tf.concat([tf.reshape(tf.multiply(tf.multiply((2.0/spice_params['Vdd']),theta_coef), Delta_2n[:,0]), [-1,1]), \
                         tf.add(tf.multiply(tf.multiply((2.0/spice_params['Vdd']),theta_coef), Delta_2n[:,1:]), \
                   tf.multiply(nn_params['Theta2n'][:,1:], tf.cast(tf.divide(spice_params['lambda'],m), dtype = tf.float32)))], 1)
    
    # Theta 1 gradient Variables are updated
theta1_grad = tf.concat([tf.reshape(tf.multiply(theta_coef, Delta_1[:,0]), [-1,1]), \
                         tf.add(tf.multiply(theta_coef, Delta_1[:,1:]), \
                   tf.multiply(nn_params['Theta1'][:,1:], tf.cast(tf.divide(spice_params['lambda'],m), dtype = tf.float32)))], 1)
theta1n_grad = tf.concat([tf.reshape(tf.multiply(theta_coef, Delta_1n[:,0]), [-1,1]), \
                         tf.add(tf.multiply(theta_coef, Delta_1n[:,1:]), \
                   tf.multiply(nn_params['Theta1n'][:,1:], tf.cast(tf.divide(spice_params['lambda'],m), dtype = tf.float32)))], 1)
    
return h_theta,[theta1_grad, theta1n_grad, theta2_grad, theta2n_grad]

In [10]:
# Random Initialization of the weights op
init_step = [
    tf.assign(nn_params['Theta1'], randInitializeWeights(input_layer_size, hidden_layer_size)),
    tf.assign(nn_params['Theta1n'], randInitializeWeights(input_layer_size, hidden_layer_size)),
    tf.assign(nn_params['Theta2'], randInitializeWeights(hidden_layer_size, num_labels)),
    tf.assign(nn_params['Theta2n'], randInitializeWeights(hidden_layer_size, num_labels))  
]

In [11]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(init_step)
sess.run(J_h, feed_dict = {X: X_train, Y: Y_train})
#x = tf.constant([0.2,0.7,1.2,1.7])
#y = tf_spiky(x)
#tf.initialize_all_variables().run()

#print(x.eval(), y.eval(), tf.gradients(y, [x])[0].eval())

2.3025827